In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv('e-shop clothing 2008.csv',sep=';')

In [ ]:
df.head()

,year,month,day,order,country,session ID,page 1 (main category),page 2 (clothing model),colour,location,model photography,price,price 2,page
0,2008,4,1,1,29,1,1,A13,1,5,1,28,2,1
1,2008,4,1,2,29,1,1,A16,1,6,1,33,2,1
2,2008,4,1,3,29,1,2,B4,10,2,1,52,1,1
3,2008,4,1,4,29,1,2,B17,6,6,2,38,2,1
4,2008,4,1,5,29,1,2,B8,4,3,2,52,1,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165474 entries, 0 to 165473
Data columns (total 1 columns):
 #   Column                                                                                                                                       Non-Null Count   Dtype 
---  ------                                                                                                                                       --------------   ----- 
 0   year;month;day;order;country;session ID;page 1 (main category);page 2 (clothing model);colour;location;model photography;price;price 2;page  165474 non-null  object
dtypes: object(1)
memory usage: 1.3+ MB


In [ ]:
# Assuming the dataframe df is already loaded

# Define state as a combination of page category and clothing model (you can modify this based on requirements)
df['state'] = df['page 1 (main category)'].astype(str) + '-' + df['page 2 (clothing model)']

# Sort by session ID and order to reconstruct user paths within each session
df = df.sort_values(by=['session ID', 'order'])

# Extract transitions for each session
transitions = []
for session_id, session_data in df.groupby('session ID'):
    session_states = session_data['state'].tolist()
    for i in range(len(session_states) - 1):
        transitions.append((session_states[i], session_states[i + 1]))

# Convert transitions to a DataFrame
transitions_df = pd.DataFrame(transitions, columns=['from_state', 'to_state'])

# Display the first few rows of the transitions dataframe
print(transitions_df.head())


  from_state to_state
0      1-A13    1-A16
1      1-A16     2-B4
2       2-B4    2-B17
3      2-B17     2-B8
4       2-B8    3-C56


In [ ]:
# Count transitions between states
transition_counts = transitions_df.groupby(['from_state', 'to_state']).size().unstack(fill_value=0)

# Convert counts to probabilities to form the transition matrix
transition_matrix = transition_counts.div(transition_counts.sum(axis=1), axis=0)

# Display the transition matrix
print("Transition Matrix:\n", transition_matrix)


Transition Matrix:
 to_state        1-A1     1-A10     1-A11     1-A12     1-A13     1-A14  \
from_state                                                               
1-A1        0.036946  0.029557  0.044828  0.031527  0.038916  0.009360   
1-A10       0.019231  0.068320  0.133603  0.128036  0.016194  0.019737   
1-A11       0.034965  0.083916  0.057589  0.093789  0.044838  0.034554   
1-A12       0.019310  0.080164  0.046811  0.067291  0.063780  0.035693   
1-A13       0.015782  0.021521  0.090387  0.024390  0.043042  0.102582   
...              ...       ...       ...       ...       ...       ...   
4-P8        0.001724  0.001724  0.001724  0.001724  0.000000  0.000000   
4-P80       0.000000  0.000000  0.006061  0.000000  0.000000  0.006061   
4-P81       0.000000  0.006849  0.013699  0.006849  0.000000  0.000000   
4-P82       0.002160  0.002160  0.012959  0.000000  0.002160  0.000000   
4-P9        0.001912  0.003824  0.001912  0.001912  0.001912  0.000000   

to_state       1-

In [ ]:
# Function to simulate the user journey based on the transition matrix
def simulate_markov_chain(transition_matrix, start_state, steps=10):
    current_state = start_state
    states_sequence = [current_state]

    for _ in range(steps):
        if current_state in transition_matrix.index:
            next_state = np.random.choice(
                transition_matrix.columns,
                p=transition_matrix.loc[current_state].values
            )
            states_sequence.append(next_state)
            current_state = next_state
        else:
            break  # Stop if no outgoing transitions for the state

    return states_sequence

# Example: Simulate user journey starting from a specific state (e.g., '1-A13')
user_journey = simulate_markov_chain(transition_matrix, '4-P34', steps=5)
print("Simulated User Journey:", user_journey)


Simulated User Journey: ['4-P34', '2-B10', '2-B11', '2-B10', '2-B11', '2-B4']


In [ ]:
# Function to compute the steady-state distribution of the Markov chain
def compute_steady_state(transition_matrix):
    eigvals, eigvecs = np.linalg.eig(transition_matrix.T)
    steady_state = np.abs(eigvecs[:, np.isclose(eigvals, 1)])
    steady_state = steady_state / steady_state.sum()
    return steady_state

# Compute steady-state probabilities
steady_state_prob = compute_steady_state(transition_matrix.values)
print("Steady-State Probabilities:", steady_state_prob)


Steady-State Probabilities: [[5.81453289e-03]
 [9.83027577e-03]
 [1.12421442e-02]
 [8.79612092e-03]
 [6.16924745e-03]
 [3.97749884e-03]
 [1.02114790e-02]
 [5.92945092e-03]
 [6.41878973e-03]
 [4.07684703e-03]
 [1.94452723e-03]
 [1.03234821e-02]
 [2.93587230e-03]
 [7.30076437e-03]
 [3.46012274e-03]
 [2.26910830e-03]
 [1.55547387e-03]
 [2.05120450e-03]
 [2.56608170e-03]
 [2.42285860e-03]
 [2.87311794e-03]
 [3.99261253e-03]
 [7.64430617e-03]
 [3.87316690e-03]
 [7.81946876e-03]
 [3.59598533e-03]
 [4.73401775e-03]
 [3.21072674e-03]
 [2.77203038e-03]
 [2.01818180e-03]
 [4.79807111e-03]
 [2.98266705e-03]
 [3.10346906e-03]
 [9.88089421e-03]
 [2.28170347e-03]
 [4.80039900e-03]
 [4.27310061e-03]
 [1.65124710e-03]
 [9.60043990e-03]
 [7.50841325e-03]
 [7.01490271e-03]
 [7.12631312e-03]
 [8.58653072e-03]
 [8.50391597e-06]
 [6.98191847e-03]
 [1.17294856e-02]
 [7.80465478e-03]
 [7.05636838e-03]
 [9.11430634e-03]
 [5.59746412e-03]
 [7.49152348e-03]
 [7.27735666e-03]
 [7.42028214e-03]
 [7.20409046e-03]
